In [79]:
import os
import sys
sys.path.insert(0, os.path.dirname(os.getcwd()))
import pickle

import numpy as np
import torch
#import torch.cuda
import timeit
import pandas as pd
#import argparse
import itertools
from segment_SHAP.load_data import load_data # segment_SHAP
#from pickle import dump

# metric specific
from sklearn.metrics import auc as sklearn_auc

In [147]:
# settings
dataset_names = {'MP'}    #{sys.argv[1]}
predictor_names = {'randomForest'}    #{sys.argv[2]} {"randomForest", 'miniRocket', 'resNet'}
segmentation_names = {"clasp","greedygaussian", "equal", "infogain","nnsegment"} # {"clasp","greedygaussian", "equal", "infogain","nnsegment"} 
background_names =  {"average", "zero","sampling"} #{"average", "zero", "sampling"}
normalization_names = {"default", "normalized"}

metric_names = {"dummy"}

demo_mode = True
# demo
if demo_mode:
    dataset_names = {'gunpoint'}
    predictor_names = {'resNet'}
    segmentation_names = {'clasp'}
    background_names = {'zero'} #,'sampling'}
    normalization_names = {"default", "normalized"}

# optional
# # get infos about which explanations are evaluated
# datasets = list( explanations['attributions'].keys() )
# segmentations = list( explanations['attributions'][datasets[0]].keys() )
# predictors = list( explanations['attributions'][datasets[0]][segmentations[0]].keys() )
# backgrounds = list( explanations['attributions'][datasets[0]][segmentations[0]][predictors[0]].keys() )
# result_types = ['default','normalized']

In [148]:
# PATHS
datasets_folder_path = None
attributions_folder_path = None
trained_models_folder_path = None

In [149]:
cwd = os.getcwd()
if datasets_folder_path is None:
    datasets_folder_path = os.path.join(os.path.dirname(cwd), "datasets")
if attributions_folder_path is None:
    attributions_folder_path = os.path.join(os.path.dirname(cwd), "attributions")
if trained_models_folder_path is None:
    trained_models_folder_path = os.path.join(os.path.dirname(cwd), "trained_models")

# device
device = "cuda" if torch.cuda.is_available() else "cpu"

starttime = timeit.default_timer()

for dataset_name in dataset_names:
    print("Dataset: ", dataset_name)
    # loading dataset
    X_train, X_test, y_train, y_test, enc = load_data(subset='all', dataset_name=dataset_name, path=datasets_folder_path)
    if demo_mode:
        X_test, y_test = X_test[:2], y_test[:2]

    for predictor_name in predictor_names:
        print("Predictor: ", predictor_name)

        # load classifier
        classifier_filename = "_".join((predictor_name, dataset_name)) + ".pt"
        classifier = torch.load(os.path.join(trained_models_folder_path, predictor_name, classifier_filename),  map_location=torch.device(device)) # FileNotFoundError

        # load explanations
        attribution_filename = "_".join(("all_results", dataset_name, predictor_name)) + ".npy"
        explanations = np.load(os.path.join(attributions_folder_path, attribution_filename), allow_pickle=True).item() # FileNotFoundError

        data_list = []

        for key in itertools.product(segmentation_names, background_names, normalization_names, metric_names):
            segmentation_name, background_name, normalization_name, metric_name = key
            print("assessing", key)

            # load model and explanations to access
            try:
                attributions = explanations['attributions'][dataset_name][segmentation_name][predictor_name][background_name][normalization_name]
            except KeyError as error:
                print('Warning: attributions is missing keys, skipping to next ' + repr(error))
                continue

            result = 1.0

            data_list.append((dataset_name, segmentation_name, predictor_name, background_name, normalization_name, metric_name, None, result))

            print("elapsed time", (timeit.default_timer() -starttime))

# res_file_name = "demo_dict_result" if demo_mode else "dict_result"
# with open( "_".join( (res_file_name,dataset_name,classifier_name)) ,"wb") as f:
#     pickle.dump(results_dict,f)

Dataset:  gunpoint
Predictor:  resNet
assessing ('clasp', 'zero', 'normalized', 'dummy')
elapsed time 0.05957539996597916
assessing ('clasp', 'zero', 'default', 'dummy')
elapsed time 0.05960319994483143


In [259]:
def normalize_saliency(saliency):
    return (np.abs(saliency) + 1e-5) / (np.max(np.abs(saliency)) + 1e-5)

def AUC_perturb_score(A, A_pred, B, B_pred, classifier):

    preds = [A_pred]
    perturbed_sample = A
    for start_idx, end_idx in zip(start_points, end_points):
        perturbed_idxs = salient_order[start_idx:end_idx]
        perturbed_sample[perturbed_idxs] = B[perturbed_idxs]
        perturbed_sample_reshaped = perturbed_sample.reshape(1, n_channels, n_timepoints)

        try:
            perturbed_sample_pred[unique_class]["pred"] = classifier.predict_proba(perturbed_sample_reshaped)[0][sample_class_pred_idx]
        except AttributeError:
            perturbed_sample_pred[unique_class]["pred"] = np.array(classifier.predict(perturbed_sample_reshaped)[0][sample_class_pred_idx], ndmin=1)

        preds.append(perturbed_sample_pred)
    preds.append(B_pred)
    preds = (np.array(preds) - B_pred) * (A_pred - B_pred) # TODO  and fixed

    AUC_score = sklearn_auc(x=np.arange(len(preds)), y=preds) / (len(preds))





def AUC_difference(X_train, X_test, y_train, dataset_name, segmentation_name, predictor_name, classifier, explanations, n_steps=18):

    n_samples, n_channels, n_timepoints = X_test.shape
    n_steps = np.min([n_steps, n_timepoints-2])

    classes = np.unique(y_train) # TODO: load from label_mapping
    opposite_class_dict = {}
    for unique_class in classes:
        opposite_class_idxs = ~(y_train==unique_class)
        opposite_sample = np.mean(X_train[opposite_class_idxs], axis=0)
        opposite_class_dict[unique_class] = {"sample": opposite_sample, "pred": None}
        try:
            opposite_class_dict[unique_class]["pred"] = classifier.predict_proba(opposite_sample[None, :])[0]
        except AttributeError:
            opposite_class_dict[unique_class]["pred"] = np.array(classifier.predict(opposite_sample[None, :])[0], ndmin=1)
            # resnet direct and to numpy

    AUC_diff_array = np.zeros(X_test.shape[0])

    for sample_idx, sample in enumerate(X_test):
        importance = normalize_saliency(attributions[sample_idx])
        salient_order = np.argsort(importance.reshape(n_channels * n_timepoints))[::-1] # decreasing
        change_points = np.array(np.round(np.linspace(0, n_channels * n_timepoints, n_steps + 2)), dtype=int)
        start_points = change_points[:-2]
        end_points = change_points[1:-1]

        # deletion
        try:
            sample_pred = explanations['y_test_pred'][dataset_name][predictor_name][sample_idx]
        except KeyError:
            sample_pred = explanations['y_test_pred'][dataset_name][segmentation_name][predictor_name][sample_idx]
        
        sample_class_pred_idx = np.argmax(sample_pred)
        sample_class = classes[sample_class_pred_idx]
        sample_class_pred = sample_pred[sample_class_pred_idx]
        opposite_sample = opposite_class_dict[sample_class]["sample"]
        opposite_class_pred = opposite_class_dict[sample_class]["pred"][sample_class_pred_idx]

        A = sample.copy()
        A_pred = sample_class_pred
        B = opposite_sample.copy()
        B_pred = opposite_class_pred
        AUC_deletion = AUC_perturb_score(A, A_pred, B, B_pred, classifier)

        A = opposite_sample.copy()
        A_pred = opposite_class_pred
        B = sample.copy()
        B_pred = sample_class_pred
        AUC_insertion = AUC_perturb_score(A, A_pred, B, B_pred, classifier)

        AUC_diff = AUC_insertion - AUC_deletion
        AUC_diff_array[sample_idx] = AUC_diff

    mean_AUC_diff = np.mean(AUC_diff_array)
    return AUC_diff




n_steps = 18

AUC_difference(X_train, X_test, y_train, dataset_name, segmentation_name, predictor_name, classifier, explanations, n_steps=n_steps)

    





AttributeError: 'ResNetBaseline' object has no attribute 'predict'

In [150]:
column_names = ['Dataset', 'Segmentation', 'ML model', 'Background', 'Normalization', 'Perturb', 'Metric', "Result"]
df = pd.DataFrame(data=data_list, columns = column_names)

In [151]:
df

,Dataset,Segmentation,ML model,Background,Normalization,Perturb,Metric,Result
0,gunpoint,clasp,resNet,zero,normalized,dummy,None,1.0
1,gunpoint,clasp,resNet,zero,default,dummy,None,1.0


In [152]:
file_name = "_".join(("evaluation", predictor_name, dataset_name))
result_path = os.path.join(cwd, "results", file_name)
df.to_csv(result_path)

In [131]:
# df2 = df.pivot(index=["Dataset", "Segmentation", "ML model", "Background", "Normalization", "Perturb", "Metric"], columns="Result", values="Result")
# df2.to_dict(orient="index")